<a href="https://colab.research.google.com/github/acl-jianghao/Langchain-Scripts/blob/main/langchain_vectordb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai --upgrade
%pip install langchain --upgrade
%pip install tiktoken --upgrade
%pip install colab-env --upgrade
%pip install pypdf --upgrade

%pip install faiss-cpu --upgrade
%pip install chromadb --upgrade
%pip install pinecone-client --upgrade

%pip install ndg-httpsclient
%pip install pyopenssl
%pip install pyasn1

In [6]:
import os
import colab_env
colab_env.__version__

'0.2.0'

In [7]:
import openai
from langchain.document_loaders import PyPDFLoader

In [39]:
# !wget -P /content/gdrive/MyDrive/data/ -U NoSuchBrowser/1.0 https://arxiv.org/pdf/2307.00386.pdf

In [5]:
!ls -alt /content/gdrive/MyDrive/data/

total 475
-rw------- 1 root root 486144 Jul  4 01:25 2307.00386.pdf


In [8]:
# refer to:

loader = PyPDFLoader('/content/gdrive/MyDrive/data/2307.00386.pdf')
docs = loader.load()

In [9]:
print("There are %s document(s)."% len(docs))
print("There are %s characters in first document(page)." % len(docs[0].page_content))

There are 8 document(s).
There are 4897 characters in first document(page).


In [10]:
# refer to:

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
split_docs = text_splitter.split_documents(docs)

In [11]:
print("Now the PDF file splitted in %s documents." % len(split_docs))
print("There are %s characters in first splitted document." % len(split_docs[0].page_content))

Now the PDF file splitted in 40 documents.
There are 999 characters in first splitted document.


In [58]:
# openai.api_key = os.getenv('OPENAI_API_KEY')

In [12]:
from langchain.vectorstores import FAISS, Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

import pinecone

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [14]:
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))

Local vector index, sample codes for Faiss, Chroma an Pincone vector db.

In [41]:
index_root_dir = '/content/gdrive/MyDrive/index/'

# db_faiss = FAISS.from_documents(split_docs, embeddings)
# db_faiss.save_local(index_root_dir+'faiss_index')


# db_chroma = Chroma.from_documents(
#     split_docs, embeddings, persist_directory=index_root_dir+'chroma_index')
# db_chroma.persist()


pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),  # find at app.pinecone.io
    environment=os.getenv('PINECONE_ENV'),  # next to api key in console
)

pinecone_indexes = pinecone.list_indexes()
if 'test-index' in pinecone_indexes:
  print("do something like: pinecone.delete_index('test-index'), and re-create")
else:
  pinecone.create_index('test-index', dimension=1536, metric="cosine")
#free account limitation'[:32]'
db_pinecone = Pinecone.from_documents(split_docs, embeddings, index_name='test-index')


do something like: pinecone.delete_index('test-index'), and re-create


In [54]:
index_faiss = FAISS.load_local(index_root_dir+'faiss_index', embeddings)
index_chroma = Chroma(persist_directory=index_root_dir+'chroma_index', embedding_function=embeddings)
# index_pinecone = pinecone.Index('test-index')

In [53]:
question = "What is the density dependent model?"
answer_docs_faiss = index_faiss.similarity_search(question)
answer_docs_chroma = index_chroma.similarity_search(question)
answer_docs_pinecone = db_pinecone.similarity_search(question)

In [55]:
print(len(answer_docs_faiss), answer_docs_faiss)
print(len(answer_docs_chroma), answer_docs_chroma)
print(len(answer_docs_pinecone), answer_docs_pinecone)

4 [Document(page_content='m∗\nb,d=mb,d−gσb,dσ0, (11)\nand\nµ∗\nb,d=µb,d−gωb,dω0−gρb,dI3b,dρ03−gϕb,dϕ0−Σr,(12)\n(13)\nrespectively, with I3b,d, the isospin projection. The baryon\ndensity is given by\nnb,d=γb,dZd3k\n(2π)3, (14)\nand the scalar density is\nns\nb,d=γb,dZd3k\n(2π)3m∗\nb,d\nEb,d, (15)\nwhere γb,dis the particle degeneracy (2 for spin 1/2 particles\nand 4 for spin 3/2 particles) and Eb,d=q\nk2+m∗2\nb,dis the\nsingle particle energy.\n2.2 Density Dependent Quark Model\nIn this work, we assume that the deconfined matter phase in\nthe inner core of the compact star is composed of electrons (e)\nand quarks (up ( u), down ( d), and strange ( s)). In this phase,the matter will be described by the DDQM model expressed\nas\nmi=mi0+mI, (16)\nwhere mi0(i=u, d, s ) is the current quark mass and mIis\nthe density dependent quantity which includes the interac-\ntions of the quarks. The motivation for the DDQM approach\nis to include the interactions between quarks in a simple way.\nIt is

Question and answering

In [73]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

#default value:
# OpenAI: 'text-davinci-003'
# ChatOpenAI: 'gpt-3.5-turbo'
llm = OpenAI(temperature=0, openai_api_key=os.getenv('OPENAI_API_KEY'), model='text-davinci-003')

#chain_type:
chain = load_qa_chain(llm, chain_type='stuff', verbose=True)

In [62]:
#index: index_faiss, index_chroma, db_pinecone
def quest(question, index=index_faiss):
    docs = index.similarity_search(question)
    return chain.run(input_documents=docs, question=question)

In [74]:
quest("What is the density dependent model?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

m∗
b,d=mb,d−gσb,dσ0, (11)
and
µ∗
b,d=µb,d−gωb,dω0−gρb,dI3b,dρ03−gϕb,dϕ0−Σr,(12)
(13)
respectively, with I3b,d, the isospin projection. The baryon
density is given by
nb,d=γb,dZd3k
(2π)3, (14)
and the scalar density is
ns
b,d=γb,dZd3k
(2π)3m∗
b,d
Eb,d, (15)
where γb,dis the particle degeneracy (2 for spin 1/2 particles
and 4 for spin 3/2 particles) and Eb,d=q
k2+m∗2
b,dis the
single particle energy.
2.2 Density Dependent Quark Model
In this work, we assume that the deconfined matter phase in
the inner core of the compact star is composed of electrons (e)
and quarks (up ( u), down ( d), and strange ( s)). In this phase,the matter will be described by the DDQM model expressed
as
mi=mi0+mI, (16)
where mi0(i=u, d, s ) is the 

' The density dependent model is a model that includes the interactions between quarks in a simple way by assuming that the deconfined matter phase in the inner core of the compact star is composed of electrons (e) and quarks (up ( u), down ( d), and strange ( s)). The model is expressed as mi=mi0+mI, where mi0(i=u, d, s ) is the current quark mass and mIis the density dependent quantity which includes the interactions of the quarks.'

In [75]:
quest("What is the density dependent model?", index_chroma)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

m∗
b,d=mb,d−gσb,dσ0, (11)
and
µ∗
b,d=µb,d−gωb,dω0−gρb,dI3b,dρ03−gϕb,dϕ0−Σr,(12)
(13)
respectively, with I3b,d, the isospin projection. The baryon
density is given by
nb,d=γb,dZd3k
(2π)3, (14)
and the scalar density is
ns
b,d=γb,dZd3k
(2π)3m∗
b,d
Eb,d, (15)
where γb,dis the particle degeneracy (2 for spin 1/2 particles
and 4 for spin 3/2 particles) and Eb,d=q
k2+m∗2
b,dis the
single particle energy.
2.2 Density Dependent Quark Model
In this work, we assume that the deconfined matter phase in
the inner core of the compact star is composed of electrons (e)
and quarks (up ( u), down ( d), and strange ( s)). In this phase,the matter will be described by the DDQM model expressed
as
mi=mi0+mI, (16)
where mi0(i=u, d, s ) is the 

' The density dependent model is the DDQM model, which is expressed as mi=mi0+mI, where mi0(i=u, d, s ) is the current quark mass and mIis the density dependent quantity which includes the interactions of the quarks.'

In [70]:
quest("what is the conclusion of this document?")

' The conclusion of this document is that the properties of hybrid stars, such as the EoS, cs, γ, Mmax, Rmax and Yi, can be determined using the Maxwell method with a sharp hadron-quark phase transition.'

In [72]:
quest("what is the conclusion of this document?", index_chroma)

' The conclusion of this document is that the hybrid EoS developed is within the 2 M ⊙mass constraint Annala et al. (2018).'

Document summarize

In [76]:
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [80]:
#alternative way to spicify 'openai_api_key'
openai.api_key=os.getenv('OPENAI_API_KEY')

In [83]:
# playground: https://platform.openai.com/playground/p/default-summarize?mode=complete

llm = OpenAI(temperature=0, model_name='text-davinci-003')
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)

In [84]:
# Openai token limit 4097
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)


chain.run(input_documents=split_docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"MNRAS 000, 1–8 (2023) Preprint 4 July 2023 Compiled using MNRAS L ATEX style file v3.0
Hybrid Stars Built with Density Dependent Models
A. Issifu1⋆, F. M. da Silva1and D. P. Menezes1
1Departamento de F´ ısica - CFM - Universidade Federal de Santa Catarina Florian´ opolis - SC - CP. 476 - CEP 88.040 - 900 - Brazil
4 July 2023
ABSTRACT
Using a density dependent quark model and a relativistic model within the mean-field approximation for hadrons
with density dependent meson-baryon couplings, we construct an equation of state to describe a hybrid neutron star
consisting of nucleons and exotic baryons (hyperons and ∆-resonances). We do the study using a Maxwell construction.
The quark-hadron phase transition in the stellar matter is determined through; the structure, composition, and
properties of the hybrid neutron star matter. The macro

' This paper examines the structure, composition, and properties of hybrid neutron stars, which are composed of both nucleons and exotic baryons. Using a density dependent quark model and a relativistic model within the mean-field approximation, the authors construct an equation of state and use a Maxwell construction to determine the quark-hadron phase transition in the stellar matter. The research also covers topics such as the effects of strong magnetic fields on the equation of state of nuclear matter, the structure of neutron stars, and the properties of hadronic matter. The results of the model discussed accommodate the description of PSR J0740+6620 with hyperons, delta particles, and quark matter in its core, which is within the 2 M⊙ constraints imposed on NSs.'